In [1]:
import numpy as np
import pandas as pd
%matplotlib inline

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

#Save this for when we need to submit data back to kaggle
test_df_ids = test_df.loc[0:,"PassengerId"]
#drop data
train_df = train_df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1)
test_df = test_df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1)

#Add random seed so we get reproducible results.
seed = 7
np.random.seed(seed)
train_df.head(n=5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [2]:
train_df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [3]:
train_df.count(), test_df.count()

(Survived    891
 Pclass      891
 Sex         891
 Age         714
 SibSp       891
 Parch       891
 Fare        891
 Embarked    889
 dtype: int64, Pclass      418
 Sex         418
 Age         332
 SibSp       418
 Parch       418
 Fare        417
 Embarked    418
 dtype: int64)

fill the missing data with the mean value

In [4]:
missing_values = ["Age", "Fare"]
for i in missing_values:
    mean = train_df.loc[:, i].mean()
    train_df.loc[:, i].fillna(mean, inplace =True)
    test_df.loc[:, i].fillna(mean, inplace =True)
#Non int case for embarked, only 2 values missing here so we drop them
train_df = train_df.dropna(axis = 'index')

In [5]:
train_df["Age"].head(n=6)

0    22.000000
1    38.000000
2    26.000000
3    35.000000
4    35.000000
5    29.699118
Name: Age, dtype: float64

In [6]:
train_df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,889.000000,889.000000,889.000000,889.000000,889.000000,889.000000
mean,0.382452,2.311586,29.653446,0.524184,0.382452,32.096681
std,0.486260,0.834700,12.968366,1.103705,0.806761,49.697504
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.895800
50%,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
train_df = pd.get_dummies(train_df, columns = ["Sex", "Pclass","Embarked"])
test_df = pd.get_dummies(test_df, columns = ["Sex", "Pclass","Embarked"])
train_df.head(n=5)

,Survived,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,0,22.0,1,0,7.2500,0,1,0,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,1,0,0,1,0,0
2,1,26.0,0,0,7.9250,1,0,0,0,1,0,0,1
3,1,35.0,1,0,53.1000,1,0,1,0,0,0,0,1
4,0,35.0,0,0,8.0500,0,1,0,0,1,0,0,1


In [8]:
train_df.count()

Survived      889
Age           889
SibSp         889
Parch         889
Fare          889
Sex_female    889
Sex_male      889
Pclass_1      889
Pclass_2      889
Pclass_3      889
Embarked_C    889
Embarked_Q    889
Embarked_S    889
dtype: int64

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

train, validate = train_test_split(train_df, test_size = 0.25, random_state = seed)
x_train = train.iloc[:,1:]
y_train = train.iloc[:,0]
x_val = validate.iloc[:,1:]
y_val = validate.iloc[:,0]

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(test_df)

x_train.size/12

666.0

Time to make our ANN.
For the input layer we use the number of features, the hidden layer will be the mean of the input layer and the output layer.
For starters we will only apply one hidden layer as multiple layers increase training difficulty significantly and I fear we do not have enough data for that. 

Source: https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline


#Lets make a function that changes our probabilitys into 0 or 1
def classifyBinary(predictions, requiredprobability):
    for i in range(len(predictions)):
        if predictions[i] > requiredprobability:
            predictions[i] = 1
        else:
            predictions[i]= 0
    return predictions


def test_accuracy(model):
    y_pred = model.predict(x_val)
    y_pred = classifyBinary(y_pred, 0.5)
    cm = confusion_matrix(y_pred, y_val)
    print(cm)
    print(1-(cm[0,1]+cm[1,0])/(cm[0,0]+cm[1,1]))
    
def Create_model():
    model = Sequential()
    #Create input layer and first hidden layer
    model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))

    #Create output layer
    model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

    #Compile and fit the ANN
    model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ['accuracy'])
    #model.fit(x_train, y_train, epochs = 1000, batch_size = 666)
    return model

def create_model(Epochs, Batch_size):
    model = Sequential()
    #Create input layer and first hidden layer
    model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    #Create output layer
    model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

    #Compile and fit the ANN
    model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ['accuracy'])
    model.fit(x_train, y_train, epochs = Epochs, batch_size = Batch_size, verbose = 0)
    return model

def validate_model(Epochs, Batch_size):
    classifier = KerasClassifier(build_fn=Create_model, epochs = Epochs, batch_size = Batch_size, verbose = 0)
    kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)
    results = cross_val_score(classifier, x_train, y_train, cv=kfold)
    print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    
def create_validate_model(Epochs, Batch_size):
    validate_model(Epochs, Batch_size)
    return create_model(Epochs, Batch_size)

Using TensorFlow backend.


In [ ]:
import time
Tstart = time.perf_counter()
model = create_validate_model(500, 50)
Tstop = perf_counter()
print(Tstop)

In [ ]:
test_accuracy(model) #Test on test set

Test set and cross-validation results differ significantly.

In [ ]:
predictions = model.predict(x_test)

for i in range(len(predictions)):
    if predictions[i] > 0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0
        
predictions = np.squeeze(np.asarray(predictions.astype(int)))

In [ ]:
data_to_submit = pd.DataFrame({
    'PassengerId':test_df_ids,
    'Survived':predictions
})
data_to_submit.to_csv("./results/ANN_results.csv", index=False)


Testing this out gives us a 77,9% accuracy on the kaggle test set, not bad for a first run.
Lets tune the network and see what happens.

In [ ]:
def create_model(x_train, y_train, epochs, batch_size):
    model = Sequential()
    #Create input layer and first hidden layer
    model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 12))
    
    #Add more hidden layers
    model.add(Dense(units = 3, kernel_initializer = 'uniform', activation = 'relu'))
    
    #Create output layer
    model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

    #Compile and fit the ANN
    model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ['accuracy'])
    model.fit(x_train, y_train, epochs = 1000, batch_size = 666, verbose = 0)
    return model

#model = create_model(x_train, y_train, 1000, 666)
#y_pred = model.predict(x_val)
